<a href="https://colab.research.google.com/github/hruthiksiva/DataScience-Competitions/blob/main/enigma_round2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kaggle API

In [2]:
!gdown https://drive.google.com/u/0/uc?id=1XcMvwBxCa8WDRBbA-UsTK4T8gAyA42SP
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c consumer-complaints

Downloading...
From: https://drive.google.com/u/0/uc?id=1XcMvwBxCa8WDRBbA-UsTK4T8gAyA42SP
To: /content/kaggle.json
100% 70.0/70.0 [00:00<00:00, 123kB/s]
 73% 51.0M/70.0M [00:00<00:00, 130MB/s]
100% 70.0M/70.0M [00:00<00:00, 177MB/s]
  0% 0.00/1.81M [00:00<?, ?B/s]
100% 1.81M/1.81M [00:00<00:00, 123MB/s]
 93% 149M/159M [00:00<00:00, 174MB/s]
100% 159M/159M [00:00<00:00, 191MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  SAMPLE_SUBMISSION.csv.zip
  inflating: SAMPLE_SUBMISSION.csv   

3 archives were successfully processed.


#Imports

In [ ]:
import pandas as pd
import numpy as np

# Data Preprocessing

In [ ]:
df=pd.read_csv('train.csv',parse_dates=['Date received','Date sent to company'])
df2=pd.read_csv('test.csv',parse_dates=['Date received','Date sent to company'])
df=df.fillna('NaN')
df2=df2.fillna('NaN')
df.drop(columns=['Unnamed: 0'], inplace=True)
df2.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
#df.info()
#df.head()
#df.describe()
#df.isnull().sum()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   Unnamed: 0                    1048575 non-null  int64         
 1   Complaint_ID                  1048575 non-null  int64         
 2   Date received                 1048575 non-null  datetime64[ns]
 3   Product                       1048575 non-null  object        
 4   Sub-product                   1048575 non-null  object        
 5   Issue                         1048575 non-null  object        
 6   Sub-issue                     1048575 non-null  object        
 7   Consumer complaint narrative  1048575 non-null  object        
 8   Company public response       1048575 non-null  object        
 9   Company                       1048575 non-null  object        
 10  State                         1048575 non-null  object        
 11

In [ ]:
df2.isnull().any()

Unnamed: 0                      False
Complaint_ID                    False
Date received                   False
Product                         False
Sub-product                     False
Issue                           False
Sub-issue                       False
Consumer complaint narrative    False
Company public response         False
Company                         False
State                           False
ZIP code                        False
Tags                            False
Consumer consent provided?      False
Submitted via                   False
Date sent to company            False
Company response to consumer    False
Consumer disputed?              False
dtype: bool

In [ ]:
df.head()

,Unnamed: 0,Complaint_ID,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Consumer disputed?,Timely response?
0,0,2535701,6/8/2017,Mortgage,Conventional home mortgage,Struggling to pay mortgage,NaN,NaN,NaN,"Caliber Home Loans, Inc.",FL,34744,NaN,NaN,Fax,6/12/2017,Closed with explanation,NaN,1
1,1,3314889,7/22/2019,Credit card or prepaid card,General-purpose credit card or charge card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,On XX/XX/2019 all of my credit and debit cards...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,CA,900XX,NaN,Consent provided,Web,7/22/2019,Closed with explanation,NaN,1
2,2,3612478,4/18/2020,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I was made aware of a negative account and inq...,NaN,"EQUIFAX, INC.",CA,945XX,NaN,Consent provided,Web,4/18/2020,Closed with explanation,NaN,1
3,3,2069704,8/18/2016,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",TX,77014,NaN,NaN,Phone,8/19/2016,Closed with monetary relief,No,1
4,4,3354927,8/27/2019,Debt collection,Auto debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,Bridgecrest Acceptance Corporation,PA,19143,Servicemember,Consent not provided,Web,8/27/2019,Closed with explanation,NaN,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   Unnamed: 0                    1048575 non-null  int64         
 1   Complaint_ID                  1048575 non-null  int64         
 2   Date received                 1048575 non-null  datetime64[ns]
 3   Product                       1048575 non-null  int64         
 4   Sub-product                   1048575 non-null  int64         
 5   Issue                         1048575 non-null  object        
 6   Sub-issue                     1048575 non-null  object        
 7   Consumer complaint narrative  1048575 non-null  object        
 8   Company public response       1048575 non-null  int64         
 9   Company                       1048575 non-null  object        
 10  State                         1048575 non-null  int64         
 11

### Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
listing=['Consumer disputed?','Company response to consumer',
         'Consumer consent provided?','Submitted via','Tags','State','Company public response',
         #'Issue','Sub-issue',
         'Product','Sub-product']
for i in listing:
    le = LabelEncoder()
    le.fit(df[i].append(df2[i]))
    df[i] = le.transform(df[i])
    df2[i] = le.transform(df2[i])

le = LabelEncoder()
df['Timely response?'] = le.fit_transform(df['Timely response?'])

In [ ]:
def create_features(dfs):
    """
    Creates time series features from datetime index
    """
    df= pd.DataFrame()
    df['hour'] = dfs.dt.hour
    df['dayofweek'] = dfs.dt.dayofweek
    df['quarter'] = dfs.dt.quarter
    df['month'] = dfs.dt.month
    df['year'] = dfs.dt.year
    df['dayofyear'] = dfs.dt.dayofyear
    df['dayofmonth'] = dfs.dt.day
    df['weekofyear'] = dfs.dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    return X

In [ ]:
df[['hour1','dayofweek1','quarter1','month1','year1','dayofyear1','dayofmonth1','weekofyear1']]=create_features(df['Date received'])
df[['hour2','dayofweek2','quarter2','month2','year2','dayofyear2','dayofmonth2','weekofyear2']]=create_features(df['Date sent to company'])
df.drop(columns=['Date received','Date sent to company'],inplace=True)
df2[['hour1','dayofweek1','quarter1','month1','year1','dayofyear1','dayofmonth1','weekofyear1']]=create_features(df2['Date received'])
df2[['hour2','dayofweek2','quarter2','month2','year2','dayofyear2','dayofmonth2','weekofyear2']]=create_features(df2['Date sent to company'])
df2.drop(columns=['Date received','Date sent to company'],inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


In [ ]:
df2.tail()

,Unnamed: 0,Complaint_ID,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Consumer disputed?
459689,459689,342950,2013-03-06,10,9,"Loan modification,collection,foreclosure",NaN,NaN,11,"SELECT PORTFOLIO SERVICING, INC.",54,75119,0,3,5,2013-03-07,1,1
459690,459690,2599961,2017-08-09,4,67,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,11,Alliance Data Card Services,35,28226,0,0,5,2017-08-09,1,0
459691,459691,3824572,2020-09-01,6,15,Improper use of your report,Reporting company used your report improperly,I'm contacting you about the credit reporting ...,11,LEXISNEXIS,57,236XX,0,1,5,2020-09-10,1,0
459692,459692,1418953,2015-06-12,10,52,"Loan servicing, payments, escrow account",NaN,NaN,11,"Seterus, Inc.",10,06040,0,3,4,2015-06-17,1,1
459693,459693,3532501,2020-02-14,7,33,Attempts to collect debt not owed,Debt is not yours,I received information from you attempting to ...,11,Professional Bureau of Collections of Maryland...,20,60543,0,1,5,2020-02-14,1,0


In [ ]:
print(df['Sub-product'].describe())

# Data Visualization

## Uniqueness

In [ ]:
#print(df['Consumer disputed?'].unique())
#print(df['Company response to consumer'].unique())
print(df['Date received'].unique())


['6/8/2017' '7/22/2019' '4/18/2020' ... '9/8/2013' '6/16/2018' '3/9/2014']


In [ ]:
df.isnull().sum()

Unnamed: 0                           0
Complaint_ID                         0
Date received                        0
Product                              0
Sub-product                     100901
Issue                                0
Sub-issue                       282519
Consumer complaint narrative    737766
Company public response         601927
Company                              0
State                            21018
ZIP code                         93734
Tags                            919298
Consumer consent provided?      367626
Submitted via                        0
Date sent to company                 0
Company response to consumer         1
Consumer disputed?              719432
Timely response?                     0
dtype: int64

##Correlation


In [ ]:
print(df['']. corr(df['']))

In [ ]:
df.columns

Index(['Complaint_ID', 'Date received', 'Product', 'Sub-product', 'Issue',
       'Sub-issue', 'Consumer complaint narrative', 'Company public response',
       'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Consumer disputed?', 'Timely response?'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   Complaint_ID                  1048575 non-null  int64         
 1   Date received                 1048575 non-null  datetime64[ns]
 2   Product                       1048575 non-null  int64         
 3   Sub-product                   1048575 non-null  int64         
 4   Issue                         1048575 non-null  object        
 5   Sub-issue                     1048575 non-null  object        
 6   Consumer complaint narrative  1048575 non-null  object        
 7   Company public response       1048575 non-null  int64         
 8   Company                       1048575 non-null  object        
 9   State                         1048575 non-null  int64         
 10  ZIP code                      1048575 non-null  object        
 11

# Train Test Split

In [ ]:
y = df.loc[:,'Timely response?']
cols = [ 'Product', 'Sub-product','Company public response','State','Tags', 'Consumer consent provided?',
       'Submitted via',  'Company response to consumer',
       'Consumer disputed?']
cols1 = [ 'Product', 'Sub-product','Company public response','State','Tags', 'Consumer consent provided?',
       'Submitted via',  'Company response to consumer',
       'Consumer disputed?','hour1','dayofweek1','quarter1','month1','year1','dayofyear1','dayofmonth1','weekofyear1','hour2','dayofweek2','quarter2','month2','year2','dayofyear2','dayofmonth2','weekofyear2']
X = df[cols1]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Models

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838860 entries, 327274 to 875688
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Date received                 838860 non-null  datetime64[ns]
 1   Product                       838860 non-null  int64         
 2   Sub-product                   838860 non-null  int64         
 3   Company public response       838860 non-null  int64         
 4   State                         838860 non-null  int64         
 5   Tags                          838860 non-null  int64         
 6   Consumer consent provided?    838860 non-null  int64         
 7   Submitted via                 838860 non-null  int64         
 8   Date sent to company          838860 non-null  datetime64[ns]
 9   Company response to consumer  838860 non-null  int64         
 10  Consumer disputed?            838860 non-null  int64         
dtypes: datet

# Prediction

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')
#

0.9693429167751532

# Submission

In [ ]:
tdf=pd.read_csv('test.csv')

## Submission File